In [1]:
import torch
import os
import numpy as np
import random 

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

SEED = 31
seed_everything(SEED)

import flair
flair.device = torch.device('cuda:3') 

In [2]:
from oracle import HybridOracle
from active_learner import ActiveLearner

In [3]:
EXPERIMENT = 'absolute_sampling_small_bs'

In [4]:
oracle = HybridOracle(
    experiment_name=EXPERIMENT,
    all_data_file='../twitter_dataset/train_10.csv',
    valid_file='../twitter_dataset/valid.csv',
    test_file='../twitter_dataset/test.csv'
)

../twitter_dataset/train_10.csv
Starting with labelled data


In [5]:
sentences = oracle.get_all_sentences()

In [6]:
learner = ActiveLearner(sentences,
                        experiment_name=EXPERIMENT,
                        oracle=oracle, 
                        embeddings_storage_mode='gpu')

In [7]:
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.data import Dictionary, Sentence

In [8]:
word_embeddings = [WordEmbeddings('glove'),

                   # comment in flair embeddings for state-of-the-art results
                    FlairEmbeddings('news-forward'),
                    FlairEmbeddings('news-backward'),
                   ]

# 4. initialize document embedding by passing list of word embeddings
# Can choose between many RNN types (GRU by default, to change use rnn_type parameter)
document_embeddings: DocumentRNNEmbeddings = DocumentRNNEmbeddings(word_embeddings,
                                                                     hidden_size=512,
                                                                     reproject_words=True,
                                                                     reproject_words_dimension=256,
                                                                     )

label_dict = Dictionary()
label_dict.idx2item = [b'0', b'1']
label_dict.item2idx = {b'0': 0, b'1': 1}
# 5. create the text classifier
classifier = TextClassifier(document_embeddings, label_dictionary=label_dict)

In [9]:
classifier = classifier.load('../twitter_dataset/0_1/best-model.pt')

2019-10-19 13:35:40,158 loading file ../twitter_dataset/0_1/best-model.pt


In [10]:
classifier, opt_state = learner.step(classifier, step_num=0, max_sample_size=100000, labelling_step_size=32, step_lr=1e-5, sampling_method='absolute')

STARTING LABELLING
DONE 1/32 LABELS
DONE 2/32 LABELS
DONE 3/32 LABELS
DONE 4/32 LABELS
DONE 5/32 LABELS
DONE 6/32 LABELS
DONE 7/32 LABELS
DONE 8/32 LABELS
DONE 9/32 LABELS
DONE 10/32 LABELS
DONE 11/32 LABELS
DONE 12/32 LABELS
DONE 13/32 LABELS
DONE 14/32 LABELS
DONE 15/32 LABELS
DONE 16/32 LABELS
DONE 17/32 LABELS
DONE 18/32 LABELS
DONE 19/32 LABELS
DONE 20/32 LABELS
DONE 21/32 LABELS
DONE 22/32 LABELS
DONE 23/32 LABELS
DONE 24/32 LABELS
DONE 25/32 LABELS
DONE 26/32 LABELS
DONE 27/32 LABELS
DONE 28/32 LABELS
DONE 29/32 LABELS
DONE 30/32 LABELS
DONE 31/32 LABELS
DONE 32/32 LABELS
2019-10-19 13:36:00,978 Reading data from absolute_sampling_small_bs
2019-10-19 13:36:00,979 Train: absolute_sampling_small_bs/labelled_32_1571492160.csv
2019-10-19 13:36:00,981 Dev: absolute_sampling_small_bs/valid.csv
2019-10-19 13:36:00,985 Test: absolute_sampling_small_bs/test.csv
2019-10-19 13:36:01,053 ----------------------------------------------------------------------------------------------------
201

/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DocumentRNNEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type StackedEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type WordEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type FlairEmb

2019-10-19 13:37:22,721 ----------------------------------------------------------------------------------------------------
2019-10-19 13:37:22,722 Testing using best model ...
2019-10-19 13:37:22,723 loading file absolute_sampling_small_bs/0/best-model.pt
2019-10-19 13:38:42,363 0.7037	0.7038	0.7037
2019-10-19 13:38:42,365 
MICRO_AVG: acc 0.5429 - f1-score 0.7037
MACRO_AVG: acc 0.543 - f1-score 0.70375
0          tp: 3506 - fp: 1478 - fn: 1484 - tn: 3533 - precision: 0.7035 - recall: 0.7026 - accuracy: 0.5421 - f1-score: 0.7030
1          tp: 3532 - fp: 1485 - fn: 1478 - tn: 3506 - precision: 0.7040 - recall: 0.7050 - accuracy: 0.5438 - f1-score: 0.7045
2019-10-19 13:38:42,366 ----------------------------------------------------------------------------------------------------


In [11]:
for step_num in range(1, 400):
    classifier, opt_state = learner.step(classifier, step_num=step_num, optimizer_state=opt_state, max_sample_size=100000, labelling_step_size=32, step_lr=1e-5, sampling_method='absolute')

STARTING LABELLING
DONE 1/32 LABELS
DONE 2/32 LABELS
DONE 3/32 LABELS
DONE 4/32 LABELS
DONE 5/32 LABELS
DONE 6/32 LABELS
DONE 7/32 LABELS
DONE 8/32 LABELS
DONE 9/32 LABELS
DONE 10/32 LABELS
DONE 11/32 LABELS
DONE 12/32 LABELS
DONE 13/32 LABELS
DONE 14/32 LABELS
DONE 15/32 LABELS
DONE 16/32 LABELS
DONE 17/32 LABELS
DONE 18/32 LABELS
DONE 19/32 LABELS
DONE 20/32 LABELS
DONE 21/32 LABELS
DONE 22/32 LABELS
DONE 23/32 LABELS
DONE 24/32 LABELS
DONE 25/32 LABELS
DONE 26/32 LABELS
DONE 27/32 LABELS
DONE 28/32 LABELS
DONE 29/32 LABELS
DONE 30/32 LABELS
DONE 31/32 LABELS
DONE 32/32 LABELS
2019-10-19 13:38:59,536 Reading data from absolute_sampling_small_bs
2019-10-19 13:38:59,537 Train: absolute_sampling_small_bs/labelled_32_1571492339.csv
2019-10-19 13:38:59,538 Dev: absolute_sampling_small_bs/valid.csv
2019-10-19 13:38:59,538 Test: absolute_sampling_small_bs/test.csv
2019-10-19 13:38:59,611 ----------------------------------------------------------------------------------------------------
201

/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DocumentRNNEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type StackedEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type WordEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type FlairEmb

2019-10-19 13:40:22,205 ----------------------------------------------------------------------------------------------------
2019-10-19 13:40:22,206 Testing using best model ...
2019-10-19 13:40:22,209 loading file absolute_sampling_small_bs/1/best-model.pt
2019-10-19 13:41:41,822 0.7026	0.7027	0.7026
2019-10-19 13:41:41,824 
MICRO_AVG: acc 0.5416 - f1-score 0.7026
MACRO_AVG: acc 0.5416 - f1-score 0.70265
0          tp: 3561 - fp: 1544 - fn: 1429 - tn: 3467 - precision: 0.6976 - recall: 0.7136 - accuracy: 0.5450 - f1-score: 0.7055
1          tp: 3466 - fp: 1430 - fn: 1544 - tn: 3561 - precision: 0.7079 - recall: 0.6918 - accuracy: 0.5382 - f1-score: 0.6998
2019-10-19 13:41:41,841 ----------------------------------------------------------------------------------------------------
STARTING LABELLING
DONE 1/32 LABELS
DONE 2/32 LABELS
DONE 3/32 LABELS
DONE 4/32 LABELS
DONE 5/32 LABELS
DONE 6/32 LABELS
DONE 7/32 LABELS
DONE 8/32 LABELS
DONE 9/32 LABELS
DONE 10/32 LABELS
DONE 11/32 LABELS
DO

/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DocumentRNNEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type StackedEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type WordEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type FlairEmb

2019-10-19 13:43:30,696 ----------------------------------------------------------------------------------------------------
2019-10-19 13:43:30,697 Testing using best model ...
2019-10-19 13:43:30,698 loading file absolute_sampling_small_bs/2/best-model.pt
2019-10-19 13:44:42,953 0.7022	0.7023	0.7022
2019-10-19 13:44:42,955 
MICRO_AVG: acc 0.5411 - f1-score 0.7022
MACRO_AVG: acc 0.5411 - f1-score 0.7021
0          tp: 3615 - fp: 1602 - fn: 1375 - tn: 3409 - precision: 0.6929 - recall: 0.7244 - accuracy: 0.5484 - f1-score: 0.7083
1          tp: 3408 - fp: 1376 - fn: 1602 - tn: 3615 - precision: 0.7124 - recall: 0.6802 - accuracy: 0.5337 - f1-score: 0.6959
2019-10-19 13:44:42,956 ----------------------------------------------------------------------------------------------------
STARTING LABELLING
DONE 1/32 LABELS
DONE 2/32 LABELS
DONE 3/32 LABELS
DONE 4/32 LABELS
DONE 5/32 LABELS
DONE 6/32 LABELS
DONE 7/32 LABELS
DONE 8/32 LABELS
DONE 9/32 LABELS
DONE 10/32 LABELS
DONE 11/32 LABELS
DON

...

/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DocumentRNNEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type StackedEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type WordEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type FlairEmb

2019-10-19 14:07:45,886 ----------------------------------------------------------------------------------------------------
2019-10-19 14:07:45,887 Testing using best model ...
2019-10-19 14:07:45,888 loading file absolute_sampling_small_bs/10/best-model.pt
2019-10-19 14:08:58,214 0.6838	0.6839	0.6838
2019-10-19 14:08:58,218 
MICRO_AVG: acc 0.5196 - f1-score 0.6838
MACRO_AVG: acc 0.5153 - f1-score 0.6789000000000001
0          tp: 4042 - fp: 2213 - fn: 948 - tn: 2798 - precision: 0.6462 - recall: 0.8100 - accuracy: 0.5612 - f1-score: 0.7189
1          tp: 2797 - fp: 949 - fn: 2213 - tn: 4042 - precision: 0.7467 - recall: 0.5583 - accuracy: 0.4694 - f1-score: 0.6389
2019-10-19 14:08:58,219 ----------------------------------------------------------------------------------------------------
STARTING LABELLING
DONE 1/32 LABELS
DONE 2/32 LABELS
DONE 3/32 LABELS
DONE 4/32 LABELS
DONE 5/32 LABELS
DONE 6/32 LABELS
DONE 7/32 LABELS
DONE 8/32 LABELS
DONE 9/32 LABELS
DONE 10/32 LABELS
DONE 11/32

/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DocumentRNNEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type StackedEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type WordEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type FlairEmb

2019-10-19 14:10:44,296 ----------------------------------------------------------------------------------------------------
2019-10-19 14:10:44,297 Testing using best model ...
2019-10-19 14:10:44,298 loading file absolute_sampling_small_bs/11/best-model.pt


RuntimeError: DataLoader worker (pid 4856) is killed by signal: Killed. 